In [1]:
import pandas as pd
import os
import random
from tqdm import tqdm
import pickle

In [2]:
PATH_TO_LOAD = '../data/04_Merged'

merge_context_data = False
regenerate_context_data = True
sequence_augmentation = False

In [3]:
dynamic_context_var = ['KBI_speed']
static_context_var = ['car_id']

In [4]:
## create context data

def load_context(vehicle):
    df = pd.read_csv(os.path.join(PATH_TO_LOAD, vehicle + "_merged.csv"), parse_dates=['datetime'], low_memory=False)
    df_filt = df[['session','datetime', 'KBI_speed']]
    df_filt = df_filt.dropna(subset=['KBI_speed'])
    df_filt_sort = df_filt.sort_values(by=['session','datetime'])
    return df_filt_sort

vehicles = ['SEB880','SEB882','SEB883','SEB885','SEB888','SEB889']
context_data = pd.DataFrame()

if merge_context_data == True:
    for vehicle in tqdm(vehicles):
        context_curr = load_context(vehicle)
        context_curr['car_id'] = vehicle
        context_data = pd.concat([context_data, context_curr], axis=0)
    context_data.to_csv('../data/05_Interaction_Sequences/context.csv')

if regenerate_context_data == True:
    # selected_sessions = merged_df['session'].unique().tolist()

    context_data = pd.read_csv('../data/05_Interaction_Sequences/context.csv', parse_dates=['datetime'], index_col=0)
    
    vehicle_list = context_data.car_id.unique().tolist()
    vehicle_dict = {vehicle: random.randint(1, 50) for vehicle in vehicle_list}
    context_data['car_id'] = context_data['car_id'].map(vehicle_dict)
    context_data = context_data.sort_values(by=['session','datetime'])
    context_data['session'] = context_data['session'].astype(int)
    static_context_var.append('session')
    static_context = context_data[static_context_var].drop_duplicates(subset=['car_id', 'session'])
    # context_data = context_data[dynamic_context]
    # selected_context = ['KBI_speed', 'car_id']
    dynamic_context_var.extend(['session', 'datetime'])
    dynamic_context = context_data[dynamic_context_var]
    dynamic_context_var = [item for item in dynamic_context_var if item not in ['session', 'datetime']]
    dynamic_context = dynamic_context.groupby(['session', 'datetime'])[dynamic_context_var].mean().reset_index()

In [5]:
selected_sequence = pd.read_csv('../data/05_Interaction_Sequences/sequence_context.csv', parse_dates=['datetime'], index_col=0).reset_index()
selected_sequence['session'] = selected_sequence['session'].astype(int)

temp_df = selected_sequence.groupby('session').apply(lambda group: group.iloc[:-1]).reset_index(drop=True)
dynamic_context = dynamic_context[dynamic_context['session'].isin(temp_df.session.unique().tolist())]

training_sequence_context = pd.DataFrame()
for session in tqdm(selected_sequence['session'].unique().tolist()):
    selected_sequence_curr = selected_sequence[selected_sequence['session']==session]
    context_data_curr = dynamic_context[dynamic_context['session']==session]
    context_data_curr = context_data_curr[context_data_curr['datetime']<=selected_sequence_curr['datetime'].max()]
    training_sequence_context = pd.concat([training_sequence_context,context_data_curr], axis=0)


100%|██████████| 954/954 [00:49<00:00, 19.43it/s]


In [23]:
# testing_sessions = [16, 25]
# selected_sequence = selected_sequence[selected_sequence['session'].isin(testing_sessions)]
# training_sequence_context = training_sequence_context[training_sequence_context['session'].isin(testing_sessions)]

training_sequence_context_augmented = pd.DataFrame()
session_id = 0
if sequence_augmentation == True:
    for session in tqdm(selected_sequence['session'].unique().tolist()):
        selected_sequence_curr = selected_sequence[selected_sequence['session']==session].reset_index()
        context_curr = training_sequence_context[training_sequence_context['session']==session].reset_index()
        for i in range(len(selected_sequence_curr)-1, -1, -1):
            context_filt_curr = training_sequence_context[
                (training_sequence_context['datetime'] <= selected_sequence_curr.loc[i, 'datetime'])].copy()
            # context_filt_curr.loc[context_filt_curr.index, 'session_id'] = session_id
            context_filt_curr['session_id'] = session_id
            training_sequence_context_augmented = pd.concat([training_sequence_context_augmented, context_filt_curr], axis=0)
            session_id += 1
    dynamic_context = training_sequence_context_augmented
else:
    # if sequence_augmentation is set to false
    dynamic_context = training_sequence_context


with open('../data/05_Interaction_Sequences/train_sessions.pkl', 'rb') as pickle_file:
    train_sessions = pickle.load(pickle_file)

with open('../data/05_Interaction_Sequences/test_sessions.pkl', 'rb') as pickle_file:
    test_sessions = pickle.load(pickle_file)

train_context_data = dynamic_context[dynamic_context['session'].isin(train_sessions)]
test_context_data = dynamic_context[dynamic_context['session'].isin(test_sessions)]

train_static_context = static_context[static_context['session'].isin(train_sessions)]
test_static_context = static_context[static_context['session'].isin(test_sessions)]

train_context_data['session'] = train_context_data.groupby('session').ngroup()
test_context_data['session'] = test_context_data.groupby('session').ngroup()

train_context_data = train_context_data.rename(columns={'session': 'session_id'})
test_context_data = test_context_data.rename(columns={'session': 'session_id',})

train_context_data.to_csv('../datasets/sequential/carsii_timedelta_rand_seq/dynamic_context/train.csv', index=False)
test_context_data.to_csv('../datasets/sequential/carsii_timedelta_rand_seq/dynamic_context/test.csv', index=False)

train_static_context.to_csv('../datasets/sequential/carsii_timedelta_rand_seq/static_context/train.csv', index=False)
test_static_context.to_csv('../datasets/sequential/carsii_timedelta_rand_seq/static_context/test.csv', index=False)

/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_5448/1459495478.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_context_data['session'] = train_context_data.groupby('session').ngroup()
/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_5448/1459495478.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_context_data['session'] = test_context_data.groupby('session').ngroup()
